In [1]:
import tensorflow as tf
import numpy as np
import os
import PIL
import PIL.Image
import pathlib

In [2]:
# train_folder_path = pathlib.Path('D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/hei-train/left/')
# test_folder_path = pathlib.Path('D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/hei-test/left/')
train_folder_path = pathlib.Path('D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/hei-train-v4/left/')
test_folder_path = pathlib.Path('D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/hei-test-v2/left/')

In [3]:
images = list(train_folder_path.glob('*/*.jpg'))
image_count = len(images)
print(image_count)

231


In [4]:
train_batch_size = 32
img_height = 104
img_width = 104

In [5]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder_path,
  validation_split=0.2,
  subset="both",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=train_batch_size)

# train_ds= tf.keras.utils.image_dataset_from_directory(
#   train_folder_path,
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=train_batch_size)

Found 231 files belonging to 10 classes.
Using 185 files for training.
Using 46 files for validation.


In [6]:
class_names = train_ds.class_names

In [7]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_folder_path,
  image_size=(img_height, img_width),
  batch_size=100)

Found 20 files belonging to 10 classes.


In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
num_classes = 10

In [10]:
base_model = tf.keras.applications.ResNet152(weights='imagenet', include_top=False, input_shape=(104,104,3))
for layer in base_model.layers:
    layer.trainable = False

In [17]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(500, activation='relu')(x)
predictions = tf.keras.layers.Dense(10, activation = 'softmax')(x)

In [18]:
head_model = tf.keras.Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [19]:
head_model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

Epoch 1/5


6/6 [==============================] - 19s 2s/step - loss: 20.2472 - accuracy: 0.4378 - val_loss: 20.0465 - val_accuracy: 0.4783
Epoch 2/5
6/6 [==============================] - 9s 2s/step - loss: 3.1972 - accuracy: 0.8541 - val_loss: 6.4060 - val_accuracy: 0.7609
Epoch 3/5
6/6 [==============================] - 9s 2s/step - loss: 0.2167 - accuracy: 0.9784 - val_loss: 6.4471 - val_accuracy: 0.7826
Epoch 4/5
6/6 [==============================] - 9s 2s/step - loss: 0.0491 - accuracy: 0.9946 - val_loss: 4.7382 - val_accuracy: 0.8478
Epoch 5/5
6/6 [==============================] - 9s 2s/step - loss: 2.2112e-04 - accuracy: 1.0000 - val_loss: 4.4594 - val_accuracy: 0.8478


In [20]:
x, y = next(iter(test_ds))
predictions = np.argmax(head_model.predict(x), axis=-1)
labels = y

m = tf.keras.metrics.Accuracy()
m.update_state(predictions, labels)
m.result().numpy()

1/1 [==============================] - 3s 3s/step


0.6

In [77]:
base_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(104,104,3))
for layer in base_model.layers:
    layer.trainable = False

80134624/80134624 [==============================] - 3s 0us/step


In [78]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(1000, activation='relu')(x)
predictions = tf.keras.layers.Dense(10, activation = 'softmax')(x)

In [79]:
head_model = tf.keras.Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [80]:
head_model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

Epoch 1/5


6/6 [==============================] - 9s 1s/step - loss: 11.3425 - accuracy: 0.5405 - val_loss: 5.0318 - val_accuracy: 0.7826
Epoch 2/5
6/6 [==============================] - 7s 1s/step - loss: 2.5887 - accuracy: 0.8703 - val_loss: 8.2345 - val_accuracy: 0.7826
Epoch 3/5
6/6 [==============================] - 7s 1s/step - loss: 0.6527 - accuracy: 0.9784 - val_loss: 8.5436 - val_accuracy: 0.7826
Epoch 4/5
6/6 [==============================] - 7s 1s/step - loss: 0.3886 - accuracy: 0.9892 - val_loss: 6.4081 - val_accuracy: 0.7609
Epoch 5/5
6/6 [==============================] - 8s 1s/step - loss: 2.6278e-06 - accuracy: 1.0000 - val_loss: 7.3582 - val_accuracy: 0.8043


In [81]:
x, y = next(iter(test_ds))
predictions = np.argmax(head_model.predict(x), axis=-1)
labels = y

m = tf.keras.metrics.Accuracy()
m.update_state(predictions, labels)
m.result().numpy()

1/1 [==============================] - 1s 684ms/step


0.55

In [21]:
import torch
from torch import nn

In [23]:
import sys

In [30]:
sys.path.append('D:\Documentos\Polito\Thesis\Code\mobilenet_SFH')

In [53]:
import mobilenet

In [84]:
mobilenet.MobileNet(num_classes=600, sample_size=224, width_mult=1.)

MobileNet(
  (features): Sequential(
    (0): Sequential(
      (0): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Block(
      (conv1): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), groups=32, bias=False)
      (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(32, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Block(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), groups=64, bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (b

In [85]:
model = mobilenet.get_model()

In [86]:
checkpoint = torch.load("D:\Documentos\Polito\Thesis\Code\mobilenet_SFH\jester_mobilenet_2.0x_RGB_16_checkpoint40Nostro.pth",
                        map_location="cpu")

In [87]:
state_dict = checkpoint['state_dict']

In [88]:
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[14:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)

RuntimeError: Error(s) in loading state_dict for MobileNet:
	size mismatch for features.0.0.weight: copying a param with shape torch.Size([64, 3, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 3, 3, 3, 3]).
	size mismatch for features.0.1.weight: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for features.0.1.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for features.0.1.running_mean: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for features.0.1.running_var: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for features.1.conv1.weight: copying a param with shape torch.Size([64, 1, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32,